# Lab 1.2.3: Visualization Dashboard - SOLUTIONS

**Module:** 1.2 - Python for AI/ML  

This notebook contains solutions to all exercises from the Visualization Dashboard Lab.

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.calibration import calibration_curve
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')

print("Solutions Notebook for Visualization Dashboard")
print("=" * 50)

---

## Exercise: Custom Dashboard with Advanced Metrics

**Task:** Create a 2x2 dashboard with:
1. Learning Rate Finder curve
2. ROC Curve
3. Precision-Recall Curve
4. Calibration Plot

In [ ]:
# Generate sample data for all plots
np.random.seed(42)

# 1. Learning rate finder data
lrs = np.logspace(-6, 0, 100)
lr_losses = 2.0 - 0.5 * np.log10(lrs + 1e-7) + np.random.normal(0, 0.05, 100)
lr_losses[70:] += 0.1 * (np.arange(30) ** 1.5)  # Divergence at high LR

# 2. Classification predictions for ROC, PR, Calibration
n_samples = 1000
y_true = np.random.binomial(1, 0.3, n_samples)  # 30% positive

# Generate realistic probabilities
y_prob = np.zeros(n_samples)
y_prob[y_true == 1] = np.random.beta(5, 2, y_true.sum())  # Higher for positives
y_prob[y_true == 0] = np.random.beta(2, 5, (1-y_true).sum())  # Lower for negatives

print("Sample data generated:")
print(f"  Learning rate range: {lrs.min():.2e} to {lrs.max():.2e}")
print(f"  Classification samples: {n_samples}")
print(f"  Positive rate: {y_true.mean():.1%}")

In [ ]:
# SOLUTION: Complete Advanced Dashboard

fig = plt.figure(figsize=(14, 12))
gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)

# Color palette
colors = {
    'primary': '#3498db',
    'secondary': '#e74c3c',
    'success': '#2ecc71',
    'gray': '#95a5a6'
}

# Panel 1: Learning Rate Finder (top-left)
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(lrs, lr_losses, color=colors['primary'], linewidth=2)
ax1.set_xscale('log')
ax1.set_xlabel('Learning Rate (log scale)')
ax1.set_ylabel('Loss')
ax1.set_title('Learning Rate Finder', fontweight='bold')

# Find suggested LR (steepest descent point)
gradients = np.gradient(lr_losses)
suggested_idx = np.argmin(gradients[:70])  # Before divergence
suggested_lr = lrs[suggested_idx]
ax1.axvline(suggested_lr, color=colors['secondary'], linestyle='--', 
            label=f'Suggested LR: {suggested_lr:.2e}')
ax1.scatter([suggested_lr], [lr_losses[suggested_idx]], color=colors['secondary'], 
            s=100, zorder=5)
ax1.legend()

# Panel 2: ROC Curve (top-right)
ax2 = fig.add_subplot(gs[0, 1])
fpr, tpr, thresholds = roc_curve(y_true, y_prob)
roc_auc = auc(fpr, tpr)

ax2.plot(fpr, tpr, color=colors['primary'], linewidth=2, 
         label=f'ROC Curve (AUC = {roc_auc:.3f})')
ax2.plot([0, 1], [0, 1], color=colors['gray'], linestyle='--', label='Random Classifier')
ax2.fill_between(fpr, tpr, alpha=0.2, color=colors['primary'])

# Mark optimal threshold (Youden's J)
j_scores = tpr - fpr
optimal_idx = np.argmax(j_scores)
ax2.scatter([fpr[optimal_idx]], [tpr[optimal_idx]], color=colors['secondary'], 
            s=100, zorder=5, label=f'Optimal (thresh={thresholds[optimal_idx]:.2f})')

ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title('ROC Curve', fontweight='bold')
ax2.legend(loc='lower right')
ax2.set_xlim(-0.02, 1.02)
ax2.set_ylim(-0.02, 1.02)

# Panel 3: Precision-Recall Curve (bottom-left)
ax3 = fig.add_subplot(gs[1, 0])
precision, recall, pr_thresholds = precision_recall_curve(y_true, y_prob)
pr_auc = auc(recall, precision)

ax3.plot(recall, precision, color=colors['success'], linewidth=2,
         label=f'PR Curve (AUC = {pr_auc:.3f})')
ax3.fill_between(recall, precision, alpha=0.2, color=colors['success'])

# Baseline (random classifier)
baseline = y_true.mean()
ax3.axhline(baseline, color=colors['gray'], linestyle='--', 
            label=f'Baseline (P={baseline:.2f})')

ax3.set_xlabel('Recall')
ax3.set_ylabel('Precision')
ax3.set_title('Precision-Recall Curve', fontweight='bold')
ax3.legend(loc='lower left')
ax3.set_xlim(-0.02, 1.02)
ax3.set_ylim(-0.02, 1.02)

# Panel 4: Calibration Plot (bottom-right)
ax4 = fig.add_subplot(gs[1, 1])

# Compute calibration curve
prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=10)

ax4.plot([0, 1], [0, 1], color=colors['gray'], linestyle='--', label='Perfectly Calibrated')
ax4.plot(prob_pred, prob_true, color=colors['primary'], linewidth=2, 
         marker='o', markersize=8, label='Model')

# Add histogram of predictions
ax4_hist = ax4.twinx()
ax4_hist.hist(y_prob, bins=20, alpha=0.3, color=colors['primary'])
ax4_hist.set_ylabel('Count', color=colors['gray'])
ax4_hist.tick_params(axis='y', labelcolor=colors['gray'])

ax4.set_xlabel('Mean Predicted Probability')
ax4.set_ylabel('Fraction of Positives')
ax4.set_title('Calibration Plot', fontweight='bold')
ax4.legend(loc='upper left')
ax4.set_xlim(-0.02, 1.02)
ax4.set_ylim(-0.02, 1.02)

# Overall title
fig.suptitle('Advanced Model Evaluation Dashboard', fontsize=16, fontweight='bold', y=1.01)

plt.tight_layout()
plt.savefig('advanced_dashboard.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

print("\n✅ Advanced dashboard complete!")
print(f"   ROC AUC: {roc_auc:.3f}")
print(f"   PR AUC: {pr_auc:.3f}")

### Alternative: Interactive Dashboard Function

In [ ]:
def create_evaluation_dashboard(y_true, y_prob, lr_data=None, save_path=None):
    """
    Create a comprehensive model evaluation dashboard.
    
    Args:
        y_true: True binary labels
        y_prob: Predicted probabilities
        lr_data: Optional tuple of (learning_rates, losses) for LR finder
        save_path: Optional path to save figure
    
    Returns:
        Dictionary with computed metrics
    """
    fig = plt.figure(figsize=(14, 12))
    gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)
    
    metrics = {}
    
    # Panel 1: ROC Curve
    ax1 = fig.add_subplot(gs[0, 0])
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    metrics['roc_auc'] = auc(fpr, tpr)
    ax1.plot(fpr, tpr, 'b-', linewidth=2, label=f'AUC = {metrics["roc_auc"]:.3f}')
    ax1.plot([0, 1], [0, 1], 'k--')
    ax1.set_xlabel('FPR')
    ax1.set_ylabel('TPR')
    ax1.set_title('ROC Curve', fontweight='bold')
    ax1.legend()
    
    # Panel 2: Precision-Recall
    ax2 = fig.add_subplot(gs[0, 1])
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    metrics['pr_auc'] = auc(recall, precision)
    ax2.plot(recall, precision, 'g-', linewidth=2, label=f'AUC = {metrics["pr_auc"]:.3f}')
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision-Recall Curve', fontweight='bold')
    ax2.legend()
    
    # Panel 3: Calibration
    ax3 = fig.add_subplot(gs[1, 0])
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=10)
    ax3.plot([0, 1], [0, 1], 'k--')
    ax3.plot(prob_pred, prob_true, 'o-', linewidth=2)
    ax3.set_xlabel('Predicted')
    ax3.set_ylabel('Actual')
    ax3.set_title('Calibration Plot', fontweight='bold')
    
    # Panel 4: Score Distribution
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.hist(y_prob[y_true == 0], bins=30, alpha=0.6, label='Negative', density=True)
    ax4.hist(y_prob[y_true == 1], bins=30, alpha=0.6, label='Positive', density=True)
    ax4.set_xlabel('Predicted Probability')
    ax4.set_ylabel('Density')
    ax4.set_title('Score Distribution', fontweight='bold')
    ax4.legend()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    
    plt.show()
    
    return metrics

# Use the function
metrics = create_evaluation_dashboard(y_true, y_prob)
print(f"\nMetrics: {metrics}")

---

## Bonus: Attention Heatmap Visualization

Commonly used for Transformer interpretability.

In [ ]:
# Generate sample attention weights
np.random.seed(42)
tokens = ['The', 'cat', 'sat', 'on', 'the', 'mat', 'because', 'it', 'was', 'tired']
n_tokens = len(tokens)

# Create attention matrix (softmax over rows)
raw_attention = np.random.randn(n_tokens, n_tokens)
# Make "it" attend to "cat" more
raw_attention[7, 1] = 3.0
# Softmax
attention = np.exp(raw_attention) / np.exp(raw_attention).sum(axis=1, keepdims=True)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(attention, 
            xticklabels=tokens,
            yticklabels=tokens,
            cmap='YlOrRd',
            annot=True,
            fmt='.2f',
            ax=ax)

ax.set_xlabel('Key (attending to)', fontsize=12)
ax.set_ylabel('Query (from)', fontsize=12)
ax.set_title('Attention Weights Visualization', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Notice how 'it' (row 7) attends most to 'cat' (column 1)!")

---

## Key Takeaways

1. **ROC curves** show TPR vs FPR trade-off (use AUC for single metric)
2. **PR curves** are better for imbalanced datasets
3. **Calibration plots** show if predicted probabilities are reliable
4. **Attention heatmaps** visualize transformer interpretability

---

**End of Solutions**